## The notion of ResNet into a snippet of code

In [22]:
from keras.models import Sequential
from keras.layers import add
from keras.layers import Conv2D, MaxPooling2D,Activation,BatchNormalization,Dropout

stride = 1
CHANNEL_AXIS = 3

def res_layer(x ,filters,pooling = False,dropout = 0.0):
    temp = x
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)
    temp = BatchNormalization(axis = CHANNEL_AXIS)(temp)
    temp = Activation("relu")(temp)
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)

    x = add([temp,Conv2D(filters,(3,3),strides = stride,padding = "same")(x)])
    if pooling:
        x = MaxPooling2D((2,2))(x)
    if dropout != 0.0:
        x = Dropout(dropout)(x)
    x = BatchNormalization(axis = CHANNEL_AXIS)(x)
    x = Activation("relu")(x)
    return x


'''The above function takes arguments as follows:
    x       : input to the res_layer.
    filters : number of filters in each convolutional layer.
    pooling : whether to add a pooling layer (default is False).
    dropout : whether to add a dropout layer (default is No).'''

'The above function takes arguments as follows:\n    x       : input to the res_layer.\n    filters : number of filters in each convolutional layer.\n    pooling : whether to add a pooling layer (default is False).\n    dropout : whether to add a dropout layer (default is No).'

## Now that we have created our res_layer, lets create a custom resnet.

In [28]:
from keras.models import Model
from keras.layers import Input,Conv2D,BatchNormalization
from keras.layers import Activation,Dropout,Flatten,Dense


inp = Input(shape = (32,32,3))
x = inp
x = Conv2D(16,(3,3),strides = stride,padding = "same")(x)
x = BatchNormalization(axis = CHANNEL_AXIS)(x)
x = Activation("relu")(x)
x = res_layer(x,32,dropout = 0.2)
x = res_layer(x,32,dropout = 0.3)
x = res_layer(x,32,dropout = 0.4,pooling = True)
x = res_layer(x,64,dropout = 0.2)
x = res_layer(x,64,dropout = 0.2,pooling = True)
x = res_layer(x,256,dropout = 0.4)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(4096,activation = "relu")(x)
x = Dropout(0.23)(x)
x = Dense(100,activation = "softmax")(x)

resnet_model = Model(inp,x,name = "Resnet")
resnet_model.summary()

Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_87 (Conv2D)              (None, 32, 32, 16)   448         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 16)   64          conv2d_87[0][0]                  
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 32, 32, 16)   0           batch_normalization_57[0][0]     
_____________________________________________________________________________________________